### Work with answer folder: read file, save all correct answers of each code into a dictionary {code: [key]}

In [1]:
import os
import cv2
from skimage.io import imread, imshow
import numpy as np
import pandas as pd

In [2]:
fn_asm = os.listdir('./data')
fn_ans = os.listdir('./answer')

keys_dict = {}

In [3]:
def get_5choices(box5, all_answers):
    box5 = cv2.cvtColor(box5, cv2.COLOR_BGR2GRAY)

    _, thresh = cv2.threshold(box5, 0, 255, cv2.THRESH_BINARY_INV)
    cnts, hierarychy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    choices = []

    for cnt in cnts:
        area = cv2.contourArea(cnt)
        if area > 100:
            center = np.round(np.mean(cnt, axis=0)[0]).astype(int)
            choices.append(center)

    choices.sort(key=lambda x:x[1])
    
    for choice in choices:
        x = choice[0]
        if 200 < x < 300:
            all_answers.append('A')
        elif 300 < x < 400:
            all_answers.append('B')
        elif 400 < x < 500:
            all_answers.append('C')
        elif 500 < x < 600:
            all_answers.append('D')
        elif 600 < x < 700:
            all_answers.append('E')

In [4]:
def get_full_choices(image):
    crop = image[690:2800, 430:2100]
    height = 300
    width = 740
    step_ver = 40
    step_hor = 140

    start_hor = 0
    index = 0
    all_answers = []

    for col in range(2):
        start_ver = 0
        for row in range(6):
            index += 1
            get_5choices(crop[start_ver:start_ver + height, start_hor:start_hor + width], all_answers)
            start_ver = start_ver + height + step_ver
        start_hor = start_hor + width + step_hor
        
    return all_answers

In [5]:

for fn in fn_ans:
    code = fn[:-4]
    path = './answer/' + fn
    key_img = imread(path)
    correct_ans = get_full_choices(key_img)
    
    keys_dict[code] = correct_ans

### Loop through all students' sheets and compare with the keys to calculate grades

In [6]:
def grading(choices, code):
    global keys_dict
    keys = keys_dict[code]
    
    correct_answers = len([i for i,j in zip(choices, keys) if i == j])
    
    return round(correct_answers/len(choices), 2)*10

In [7]:
all_id = []
grades = []

for asm in fn_asm:
    std_id = asm[:asm.index('_')]
    rem = asm[asm.index('_') + 1:]
    fullname = rem[:rem.index('_')]
#     sname, fname = split_name(fullname)
    code = rem[rem.index('_') + 1 : rem.index('.')]
    
    path = './data/' + asm
    sheet = imread(path)
    full_choices = get_full_choices(sheet)
    
    all_id.append(std_id)
    grades.append(grading(full_choices, code))

### Create dataframe to save the result: 2 columns "Student ID" and "Grades"

In [8]:
df_grades = pd.DataFrame(columns=['Student ID', 'Grades'], index=range(1, len(fn_asm)+1))
df_grades['Student ID'] = all_id
df_grades['Grades'] = grades

df_grades.to_csv('grading.csv')